In [1]:
import torch
import logging
import torch.nn as nn
from tqdm import tqdm
from pathlib import Path
from torch import autograd
from sklearn import model_selection
from torch.utils.data import Dataset, Subset, DataLoader

from strokes import StrokePatientsMIDataset
from strokesdict import STROKEPATIENTSMI_LOCATION_DICT
import scipy
from torcheeg.transforms import Select,BandSignal,Compose,ToTensor
from to import ToGrid
from typing import Callable, Dict, Union, List
import numpy as np
import soxr
from downsample import SetSamplingRate
from baseline import BaselineCorrection
from torcheeg.transforms import EEGTransform, Select,BandSignal,Compose,ToTensor

dataset = StrokePatientsMIDataset(root_path='../../mi_swin/subdataset',
                                  io_path='.torcheeg/datasets_1739351117352_GoNti',
                        chunk_size=500,  # 1 second
                        overlap = 0,
                        offline_transform=Compose(
                                [BaselineCorrection(),
                                SetSamplingRate(origin_sampling_rate=500,target_sampling_rate=128),
                                BandSignal(sampling_rate=128,band_dict={'frequency_range':[8,40]})
                                ]),
                        online_transform=Compose(
                                # [ToTensor()]),
                                [ToGrid(STROKEPATIENTSMI_LOCATION_DICT),ToTensor()]),
                
                        label_transform=Select('label'),
                        num_worker=8
)
print(dataset[0][0].shape) #EEG shape(1,30,128)
print(dataset[0][1])  # label (int)
print(len(dataset))

/root/autodl-tmp/conda/envs/law/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[2025-02-12 17:06:06] INFO (torcheeg/MainThread) 🔍 | Detected cached processing results, reading cache from .torcheeg/datasets_1739351117352_GoNti.


torch.Size([128, 9, 9])
0
160


In [2]:
import os
import shutil

def delete_folder_if_exists(target_folder_name):
    # 获取父文件夹中的所有内容
    parent_folder = os.getcwd()
    for folder_name in os.listdir(parent_folder):
        folder_path = os.path.join(parent_folder, folder_name)

        # 检查是否是文件夹并且名称是否匹配
        if os.path.isdir(folder_path) and folder_name == target_folder_name:
            try:
                # 删除目标文件夹
                shutil.rmtree(folder_path)
                print(f"已删除文件夹: {folder_path}")
            except Exception as e:
                print(f"删除文件夹 {folder_path} 时出错: {e}")

In [3]:
from model import SwinTransformer, SwinTransformer_D

HYPERPARAMETERS = {
    "seed": 42,
    "batch_size": 16,
    "lr": 1e-5,
    "weight_decay": 1e-5,
    "num_epochs": 30,
}
from torcheeg.model_selection import KFoldPerSubjectGroupbyTrial
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from torch.utils.data import DataLoader
from torcheeg.trainers import ClassifierTrainer
k_fold = KFoldPerSubjectGroupbyTrial(
    n_splits=4,
    shuffle=True,
    random_state=42)

training_metrics = []
test_metrics = []

for i, (training_dataset, test_dataset) in enumerate(k_fold.split(dataset)):
    delete_folder_if_exists(target_folder_name='lightning_logs')
    model = SwinTransformer(in_chans=128,
                            window_size=3,
                            num_classes=2,
                            embed_dim=96,
                            depths=(2, 2, 4, 2),
                            num_heads=(2, 2, 4, 6)
                            )
    trainer = ClassifierTrainer(model=model,
                                num_classes=2,
                                lr=HYPERPARAMETERS['lr'],
                                weight_decay=HYPERPARAMETERS['weight_decay'],
                                metrics=["accuracy"],
                                accelerator="gpu")
    
    training_loader = DataLoader(training_dataset,
                             batch_size=HYPERPARAMETERS['batch_size'],
                             shuffle=True)
    test_loader = DataLoader(test_dataset,
                             batch_size=HYPERPARAMETERS['batch_size'],
                             shuffle=False)
    # 提前停止回调
    early_stopping_callback = EarlyStopping(
        monitor='train_loss',
        patience=10, 
        mode='min',
        verbose=True
    )
    
    trainer.fit(training_loader,
                test_loader,
                max_epochs=HYPERPARAMETERS['num_epochs'],
                callbacks=[early_stopping_callback],
                enable_progress_bar=True,
                enable_model_summary=False,
                limit_val_batches=0.0)
    # training_result = trainer.test(training_loader,
    #                                enable_progress_bar=True,
    #                                enable_model_summary=True)[0]
    test_result = trainer.test(test_loader,
                               enable_progress_bar=True,
                               enable_model_summary=True)[0]
    # training_metrics.append(training_result["test_accuracy"])
    test_metrics.append(test_result["test_accuracy"])
     
print({
    # "training_metric_avg": np.mean(training_metrics),
    # "training_metric_std": np.std(training_metrics),
    "test_metric_avg": np.mean(test_metrics),
    "test_metric_std": np.std(test_metrics)
})

[2025-02-12 17:06:26] INFO (torcheeg/MainThread) 📊 | Create the split of train and test set.
[2025-02-12 17:06:26] INFO (torcheeg/MainThread) 😊 | Please set split_path to .torcheeg/model_selection_1739351186784_toXDa for the next run, if you want to use the same setting for the experiment.
/root/autodl-tmp/conda/envs/law/lib/python3.8/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755903507/work/aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: /root/autodl-tmp/.autodl/kinlaw/GANSER/mycode/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/root/autodl-tmp/conda/envs/law/lib/python3.8/site-pa

Epoch 0: 100%|██████████| 8/8 [00:01<00:00,  7.66it/s, loss=0.711, v_num=0, train_loss=0.679, train_accuracy=0.500]

Metric train_loss improved. New best score: 0.679


Epoch 0: 100%|██████████| 8/8 [00:01<00:00,  7.64it/s, loss=0.711, v_num=0, train_loss=0.679, train_accuracy=0.500]

[2025-02-12 17:06:31] INFO (torcheeg/MainThread) 
[Train] train_loss: 0.711 train_accuracy: 0.542 



...

Epoch 29: 100%|██████████| 8/8 [00:00<00:00,  9.11it/s, loss=0.00146, v_num=0, train_loss=0.00053, train_accuracy=1.000]

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/root/autodl-tmp/conda/envs/law/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 128 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(



Testing DataLoader 0: 100%|██████████| 3/3 [00:00<00:00, 87.50it/s] 

[2025-02-12 17:08:01] INFO (torcheeg/MainThread) 
[Test] test_loss: 1.014 test_accuracy: 0.825 



Testing DataLoader 0: 100%|██████████| 3/3 [00:00<00:00, 75.33it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_accuracy          0.824999988079071
        test_loss            1.014053463935852
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'test_metric_avg': 0.7999999970197678, 'test_metric_std': 0.08838836029237385}


In [4]:
for i, score in enumerate(test_metrics):
    print(i,score)

0 0.824999988079071
1 0.8500000238418579
2 0.699999988079071
3 0.8500000238418579
4 0.6000000238418579
5 0.6000000238418579
6 0.550000011920929
7 0.550000011920929
